In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import detrend, filtfilt, firwin, lfilter
import scipy.io
from pathlib import Path

In [ ]:
SAMPLE_RATE = 9000
FILE_DIR_INPUT = "raw_data/ball"
FILE_DIR_OUTPUT = "mat_data/filtered_FIR/ball"
EXAMPLE_PATH_MAT = "example.mat"

In [3]:
files = []
for file in Path(FILE_DIR_INPUT).iterdir():
    if file.is_file():
        files.append(file)

In [11]:
def read_data_raw(FILE_PATH_RAW):
    header = []
    data = []
    with open(FILE_PATH_RAW, 'r') as file:
        for line in file:
            line = line.strip()
            if line == "Data as Time Sequence:":
                break
            if line:  
                header.append(line)
        
        for line in file:
            line = line.strip()
            if line:  
                try:
                    value = float(line)
                    data.append(value)
                except ValueError:
                    continue  
    return data, header

In [12]:
def find_peak_frequency_in_range(signal, sampling_rate, f_min, f_max):
    n = len(signal)
    freq = np.fft.rfftfreq(n, d=1/sampling_rate)
    fft_values = np.fft.rfft(signal)
    magnitude = np.abs(fft_values)
    mask = (freq >= f_min) & (freq <= f_max)
    freq_in_range = freq[mask]
    magnitude_in_range = magnitude[mask]
    if len(magnitude_in_range) == 0:
        return None, None
    max_index = np.argmax(magnitude_in_range)
    peak_frequency = freq_in_range[max_index]
    peak_amplitude = magnitude_in_range[max_index]
    return peak_frequency, peak_amplitude

In [13]:
def fir_bandstop(numtaps, lowcut, highcut, fs):
    # numtaps — число коэффициентов (порядок фильтра + 1)
    taps = firwin(
        numtaps=numtaps,
        cutoff=[lowcut, highcut],
        fs=fs,
        pass_zero='bandstop'
    )
    return taps

In [17]:
for file in files:
    signal, header = read_data_raw(file)
    peak_freq, peak_vibr = find_peak_frequency_in_range(signal, SAMPLE_RATE, 1, 30)
    lowcut = peak_freq - 2
    highcut = peak_freq + 2
    taps = firwin(8001, [lowcut, highcut], pass_zero=True, fs=SAMPLE_RATE)
    filtered_signal = lfilter(taps, 1.0, signal)
    mat_source = scipy.io.loadmat(EXAMPLE_PATH_MAT)
    mat_source["X118_DE_time"] = np.array([[x] for x in filtered_signal])
    
    file_name = file.name.split(".")[:-1][0]
    print(str(FILE_DIR_OUTPUT) + "/" + file_name + ".mat")
    scipy.io.savemat(str(FILE_DIR_OUTPUT) + "/" + file_name + ".mat", mat_source)

mat_data/filtered_FIR/normal/30Hz_accelerometer.mat
mat_data/filtered_FIR/normal/20Hz_accelerometer.mat
mat_data/filtered_FIR/normal/40Hz_accelerometer.mat
mat_data/filtered_FIR/normal/10Hz_accelerometer.mat
mat_data/filtered_FIR/normal/50Hz_accelerometer.mat
